In [2]:
from collections import Counter
import math
from itertools import product, combinations, permutations
from operator import mul
from IPython.display import clear_output
cipher_text = "BQZRMQ KLBOXE WCCEFL DKRYYL BVEHIZ NYJQEE BDYFJO PTLOEM EHOMIC \
UYHHTS GKNJFG EHIMK NIHCTI HVRIHA RSMGQT RQCSXX CSWTNK PTMNSW \
AMXVCY WEOGSR FFUEEB DKQLQZ WRKUCO FTPLOT GOJZRI XEPZSE ISXTCT \
WZRMXI RIHALE SPRFAE FVYORI HNITRG PUHITM CFCDLA HIBKLH RCDIMT \
WQWTOR DJCNDY YWMJCN HDUWOF DPUPNG BANULZ NGYPQU LEUXOV FFDCEE \
YHQUXO YOXQUO DDCVIR RPJCAT RAQVFS AWMJCN HTSOXQ UODDAG BANURR \
REZJGD VJSXOO MSDNIT RGPUHN HRSSSF VFSINH MSGPCM ZJCSLY GEWGQT \
DREASV FPXEAR IMLPZW EHQGMG WSEIXE GQKPRM XIBFWL IPCHYM OTNXYV \
FFDCEE YHASBA TEXCJZ VTSGBA NUDYAP IUGTLD WLKVRI HWACZG PTRYCE \
VNQCUP AOSPEU KPCSNG RIHLRI KUMGFC YTDQES DAHCKP BDUJPX KPYMBD \
IWDQEF WSEVKT CDDWLI NEPZSE OPYIW".replace(" ", "")

#decrypt cipher
common_english_words = ["the", "and", "for", "are", "call", "but", "not", "way", "who", "have",
                 "you", "all", "any", "can", "had", "her", "was", "one", "our", "out", "day", 
                "get", "has", "him", "his", "how", "man", "new", "now", "old", "see", "two", "time"
                "boy", "did", "its", "let", "put", "say", "she", "too", "use", "that", "with", "such"
                "this", "will", "your", "from", "they", "know", "want", "been", "good", "much", "some"]
common_english_words = [word.upper() for word in common_english_words]


In [8]:
def find_group_freq(group_len, cipher_text):          #Kasiski examination to find repeated groups of desired length
    tempct = cipher_text
    groups = []
    length_ct = len(tempct)                           
    while length_ct > group_len:
        length_ct = len(tempct)
        group = tempct[0:group_len]
        groups.append(group)
        
        tempct = tempct[1:]
    
        
    group_freq = {}             
    for i in range(len(groups)):
        repeated_group = 0
        group_freq[groups[i]] = 0
        for j in range(len(groups)):
            
            if groups[i]==groups[j]:
                repeated_group = repeated_group +1
                group_freq[groups[i]] = repeated_group
           
        group_freq_sorted = sorted(group_freq.items(), key=lambda kv: kv[1], reverse = True)

    return group_freq_sorted


In [7]:
group_repetitions = find_group_freq(3, cipher_text)
most_repeated_group = group_repetitions[0][0]

def find_key_length(most_repeated_group):           # finding the key length by using GCD
    indices = []
    for i in range (0, len(cipher_text)):
        if most_repeated_group == cipher_text[i:i+3]:
           indices.append(i+1)
    
    gcd = indices[0]
    for i in range(1, len(indices)):
        gcd = math.gcd(gcd, indices[i])
    print(indices)
    return gcd

[80, 168, 184, 472, 504]


In [12]:
key_length = find_key_length(most_repeated_group)

def split_cipher(key_length):                   # split ciphertext into "key length" separate cipher
    temp_ct = cipher_text
    splitted_cipher = []
    while len(temp_ct)>=key_length:
        splitted_cipher.append(temp_ct[:key_length])
        temp_ct = temp_ct[key_length:]
        
    return splitted_cipher

[80, 168, 184, 472, 504]


In [17]:
cipher_list = split_cipher(key_length)            # the ciphertext is splitted into eight separate ciphers and
cipher_list                                       # the letters counted, and finally stored in the "mydict" dictionary
mydict = {}

for i in range (0, len(cipher_list)):
    for j in range (0,len(cipher_list[i])):
        if cipher_list[i][j] not in mydict:
            mydict[cipher_list[i][j]] = [0]*key_length
# for loop to add all chars to dict as dict with array
# for loop to count

for i in range (0, len(cipher_list)):
    for j in range (0,len(cipher_list[i])):
        counter = 0
        for k in range(0, len(cipher_list)):
            if cipher_list[i][j] == cipher_list[k][j]:
                counter = counter + 1
        mydict[cipher_list[i][j]][j] = counter

most_common_char = []
sortert = []
nokkel = ""
for i in range(0, key_length):
    sortert = sorted(mydict.items(), key=lambda kv: kv[1][i], reverse = True)
    nokkel = sortert[0][0]
    most_common_char.append(nokkel)
most_common_char
# mydict_s = sorted(mydict.items(), key=lambda kv: kv[0])
# mydict_s

['F', 'H', 'E', 'E', 'I', 'X', 'G', 'C']

In [6]:
mc_english_letters = [['E','T','A','O','I','N'], [12.02, 9.10, 8.12, 7.68, 7.31, 6.95]]# + SHRDLU
etaoin_combinations = product(mc_english_letters[0], repeat = 8)
min_freq = sum(mc_english_letters[1]) + 12.02 + 6.95              # mean frequency of the combinations
print(min_freq)
possible_keys = []
for x in etaoin_combinations:
    tot_freq = 0
    for i in x:
        index = mc_english_letters[0].index(i) 
        freq = mc_english_letters[1][index]
        tot_freq += freq                            # total frequency of each key
    if tot_freq > min_freq:                         # keep only keys with higher total frequency than the mean
        key = ""
        for i in range(len(most_common_char)):
            ASCII_mcc = ord(most_common_char[i])
            ASCII_char_in_x = ord(x[i])            
            offset = ASCII_mcc - ASCII_char_in_x

            if offset >= 0:
                key_char = chr(65 + offset)
            else:
                offset = 26 + offset
                key_char = chr(65 + offset)
            key += key_char
        possible_keys.append(key)
len(possible_keys)

70.15


545732

In [7]:
n = round(len(cipher_text)/key_length)                      
ranked_keys = {}
testing_key = 0                                                
for key in possible_keys:                                      #decrypt the ciphertext with every single key
    if testing_key % 10000 == 0:
        clear_output()
        print("keys tested: ", testing_key)
        
    score = 0
    testing_key += 1
    plaintxt = ""
    exp_key = mul(key, n)                #repeat the key over and over until same length as ciphertext
    for i in range(0, len(cipher_text)):
        
        ASCII_cp = ord(cipher_text[i])
        ASCII_key = ord(exp_key[i])
        offset = ASCII_cp - ASCII_key
        if offset >= 0:
            plaintxt += chr(65 + offset)
        
        else:
            plaintxt += chr(90 + offset +1)
    
    for word in common_english_words:    # look for common english words in the decrypted text and score for every finding
        if plaintxt.find(word) >= 0:
            score += 1
    
    if score > 0:                    # keep only keys with a score to shorten our possible_keys list
        ranked_keys[key] = score
ranked_keys = sorted(ranked_keys.items(), key=lambda kv: kv[1], reverse = True)

keys tested:  540000


In [8]:
for i in range(0, 3):    #decrypt the ciphertext with the 3 highest scored keys
    plaintxt = ""
    exp_key = mul(ranked_keys[i][0], n)
    for j in range(0, len(cipher_text)):
        ASCII_cp = ord(cipher_text[j])
        ASCII_key = ord(exp_key[j])
        offset = ASCII_cp - ASCII_key
        if offset >= 0:
            plaintxt += chr(65 + offset)
        else:
            plaintxt += chr(90 + offset + 1)
    print('Key :', [ranked_keys[i][0]])
    print(plaintxt)

Key : ['BDLAEJNY']
ANORIHXNALMESTPGEISKNPLNASTHEQAAINTEXULHILETHFRODEDMETHAGEISCBALEDTHEDXPHERTEYITHEPRODTSSOFCOOKERTINGGGOMPLAIOIEXTTOCJEHERTEXUXSKNOWNBHENCIPHFGINGOREORRYPTIOOGESTORIOVTHEPLAJCTEXTFRPBTHECIPITRTEXTITSECIPHESXNGORDEDGYPTIONUWEMANYSDWEMESUSFSFORENCSNPTIONCPCSTITUTFIHEAREAPUSTUDYKODWNASCRZETOGRAPINSUCHASDWEMEISKODWNASACSNPTOGRAQWICSYSTFBORACIPITRTECHNJFUESUSEEUORDECIQWERINGANTSSAGEWJIHOUTANZZNOWLEDHTOFTHEEORIPHERIOVDETAILTUALLINTPIHEAREAPUCRYPTAOPLYSISCSNPTANALZHISISWHBITHELAYQTRSONCAMASBREAKJCGTHECOETTHEAREBHOFCRYPUDGRAPHYBCDCRYPTBCALYSISUDGETHERBGECALLEERRYPTOLPVY
Key : ['BDLAEKCY']
ANORIGINALMESSAGEISKNOWNASTHEPLAINTEXTWHILETHECODEDMESSAGEISCALLEDTHECIPHERTEXTTHEPROCESSOFCONVERTINGFROMPLAINTEXTTOCIPHERTEXTISKNOWNASENCIPHERINGORENCRYPTIONRESTORINGTHEPLAINTEXTFROMTHECIPHERTEXTISDECIPHERINGORDECRYPTIONTHEMANYSCHEMESUSEDFORENCRYPTIONCONSTITUTETHEAREAOFSTUDYKNOWNASCRYPTOGRAPHYSUCHASCHEMEISKNOWNASACRYPTOGRAPHICSYSTEMORACIPHERTECHNIQUESUSEDFORDECIPHERINGAMESSAGEWITHOUTANYKNO